# 0.导入相关模块并定义全局变量

In [118]:
from os import listdir
from os.path import isfile,join
import re 
import pandas as pd
from pandas import DataFrame, Series
pd.set_option('display.max_columns', 200)

path = r'/home/duhan/data/商品期货1分钟201509csv'
filenamePattern = r'(?P<code>\D+)(?P<num>\d+).csv' 
header=['date', 'time', 'open', 'high', 'low', 'close', 'lot', 'volume']

 ## (1)定义品种目录


In [119]:
InstrumentCatalog = {
    'ZZSRX': 'ZZSR',
    'ZZSRY': 'ZZSR',
}

# 1.读取目录中的所有文件名列表

In [120]:
filenameList = [ filename for filename in listdir(path) if isfile(join(path,filename))  and re.match(filenamePattern, filename)]
filenameDataset = DataFrame(filenameList, columns=['name'])
filenameDataset['code'] = filenameDataset['name'].apply(lambda x: re.match(filenamePattern, x).group('code'))
filenameDataset['num'] = filenameDataset['name'].apply(lambda x: re.match(filenamePattern, x).group('num'))
filenameDataset['catalog'] = filenameDataset['code'].apply(lambda x: InstrumentCatalog.get(x, x))
filenameDataset.head()

,name,code,num,catalog
0,SQCU03.csv,SQCU,03,SQCU
1,SQNI11.csv,SQNI,11,SQNI
2,DLJ10.csv,DLJ,10,DLJ
3,ZZCF05.csv,ZZCF,05,ZZCF
4,DLFB11.csv,DLFB,11,DLFB


# 2. 获取所有品种列表

In [121]:
instrumentList  = list(filenameDataset.groupby('catalog').count().index)
print instrumentList
filenameDataset.groupby('catalog').count().head(10)

['DLAX', 'DLAY', 'DLB', 'DLBB', 'DLC', 'DLCS', 'DLFB', 'DLI', 'DLJ', 'DLJD', 'DLJM', 'DLL', 'DLM', 'DLP', 'DLPP', 'DLV', 'DLY', 'SQAG', 'SQAL', 'SQAU', 'SQBUX', 'SQBUY', 'SQCU', 'SQFU', 'SQHC', 'SQNI', 'SQPB', 'SQRB', 'SQRU', 'SQSN', 'SQWR', 'SQZN', 'ZZCF', 'ZZFG', 'ZZJR', 'ZZLR', 'ZZMA', 'ZZME', 'ZZOI', 'ZZPM', 'ZZRI', 'ZZRM', 'ZZRS', 'ZZSF', 'ZZSM', 'ZZSR', 'ZZTA', 'ZZTC', 'ZZWH']


,name,code,num
catalog,,,
DLAX,7,7,7
DLAY,4,4,4
DLB,5,5,5
DLBB,5,5,5
DLC,7,7,7
DLCS,6,6,6
DLFB,7,7,7
DLI,13,13,13
DLJ,11,11,11


# 3. 定义通过文名件导入数据集的方法

In [122]:
def loadDataFile(filename):
    """
    根据文件名导入数据
    """    
    num = re.match(filenamePattern, filename).group('num')
    df =  pd.read_csv( join(path, filename), names=header)
    df.index = df.apply(lambda x: '%s %s' % (x['date'], x['time']) , axis =1)
    df.columns = [ [filename[:-4]]*8, df.columns ]
    return df


In [123]:
loadDataFile('DLAX05.csv').head()

DLAX05                                          
                        date   time  open  high   low close lot volume
2015-09-01 09:01  2015-09-01  09:01  4307  4307  4305  4307  24  33522
2015-09-01 09:02  2015-09-01  09:02  4304  4305  4303  4305  36  33512
2015-09-01 09:03  2015-09-01  09:03  4303  4303  4303  4303  16  33502
2015-09-01 09:04  2015-09-01  09:04  4302  4302  4299  4299  38  33498
2015-09-01 09:05  2015-09-01  09:05  4299  4299  4295  4296  56  33494

# 4.定义导入一种品种所有数据的函数

In [124]:
def loadInstrumentData(catalog):
    """
    给定顶一个品种代码，导入该品种的所有数据文件
    """
    c1 = filenameDataset['catalog'] == catalog
    df = pd.concat([ loadDataFile(filename) for filename in filenameDataset[ c1 ]['name'] ] , axis=1)    
    df = df.swaplevel(0,1,axis=1)
    df = df.sort_index(axis=1)
    return df

In [125]:
df =loadInstrumentData('DLI').head(10)
df 

close                                                      \
                  DLI01 DLI02  DLI03 DLI04  DLI05  DLI06 DLI07 DLI08  DLI09   
2015-09-01 09:01  388.0   NaN    NaN   NaN  367.0  363.5   NaN   NaN  466.0   
2015-09-01 09:02  387.5   NaN    NaN   NaN  367.0    NaN   NaN   NaN  465.5   
2015-09-01 09:03  389.0   NaN    NaN   NaN  368.0    NaN   NaN   NaN  464.5   
2015-09-01 09:04  388.5   NaN    NaN   NaN  368.5    NaN   NaN   NaN  464.5   
2015-09-01 09:05  390.5   NaN    NaN   NaN  369.0    NaN   NaN   NaN  465.0   
2015-09-01 09:06  390.0   NaN  371.0   NaN  369.5    NaN   NaN   NaN  466.0   
2015-09-01 09:07  392.0   NaN  372.5   NaN  371.0    NaN   NaN   NaN  468.0   
2015-09-01 09:08  391.0   NaN    NaN   369  370.5    NaN   NaN   NaN  467.5   
2015-09-01 09:09  392.5   NaN    NaN   NaN  372.0    NaN   NaN   NaN  467.5   
2015-09-01 09:10  392.5   NaN    NaN   NaN  371.5    NaN   NaN   NaN  467.5   

                                                   date                    \
                  DLI10 DLI11  DLI12  DLI13       DLI01 DLI02       DLI03   
2015-09-01 09:01    NaN   NaN    NaN  385.6  2015-09-01   NaN         NaN   
2015-09-01 09:02  423.0   NaN    NaN  385.2  2015-09-01   NaN         NaN   
2015-09-01 09:03    NaN   NaN    NaN  386.6  2015-09-01   NaN         NaN   
2015-09-01 09:04    NaN   NaN    NaN  386.3  2015-09-01   NaN         NaN   
2015-09-01 09:05    NaN   NaN  392.5  388.1  2015-09-01   NaN         NaN   
2015-09-01 09:06    NaN   NaN    NaN  387.7  2015-09-01   NaN  2015-09-01   
2015-09-01 09:07    NaN   NaN    NaN  389.5  2015-09-01   NaN  2015-09-01   
2015-09-01 09:08  437.5   NaN    NaN  389.1  2015-09-01   NaN         NaN   
2015-09-01 09:09    NaN   NaN    NaN  389.7  2015-09-01   NaN         NaN   
2015-09-01 09:10    NaN   NaN    NaN  389.7  2015-09-01   NaN         NaN   

                                                                              \
                       DLI04       DLI05       DLI06 DLI07 DLI08       DLI09   
2015-09-01 09:01         NaN  2015-09-01  2015-09-01   NaN   NaN  2015-09-01   
2015-09-01 09:02         NaN  2015-09-01         NaN   NaN   NaN  2015-09-01   
2015-09-01 09:03         NaN  2015-09-01         NaN   NaN   NaN  2015-09-01   
2015-09-01 09:04         NaN  2015-09-01         NaN   NaN   NaN  2015-09-01   
2015-09-01 09:05         NaN  2015-09-01         NaN   NaN   NaN  2015-09-01   
2015-09-01 09:06         NaN  2015-09-01         NaN   NaN   NaN  2015-09-01   
2015-09-01 09:07         NaN  2015-09-01         NaN   NaN   NaN  2015-09-01   
2015-09-01 09:08  2015-09-01  2015-09-01         NaN   NaN   NaN  2015-09-01   
2015-09-01 09:09         NaN  2015-09-01         NaN   NaN   NaN  2015-09-01   
2015-09-01 09:10         NaN  2015-09-01         NaN   NaN   NaN  2015-09-01   

                                                             high        \
                       DLI10 DLI11       DLI12       DLI13  DLI01 DLI02   
2015-09-01 09:01         NaN   NaN         NaN  2015-09-01  389.0   NaN   
2015-09-01 09:02  2015-09-01   NaN         NaN  2015-09-01  388.0   NaN   
2015-09-01 09:03         NaN   NaN         NaN  2015-09-01  389.0   NaN   
2015-09-01 09:04         NaN   NaN         NaN  2015-09-01  389.0   NaN   
2015-09-01 09:05         NaN   NaN  2015-09-01  2015-09-01  390.5   NaN   
2015-09-01 09:06         NaN   NaN         NaN  2015-09-01  390.5   NaN   
2015-09-01 09:07         NaN   NaN         NaN  2015-09-01  392.5   NaN   
2015-09-01 09:08  2015-09-01   NaN         NaN  2015-09-01  392.0   NaN   
2015-09-01 09:09         NaN   NaN         NaN  2015-09-01  392.5   NaN   
2015-09-01 09:10         NaN   NaN         NaN  2015-09-01  393.0   NaN   

                                                                             \
                  DLI03 DLI04  DLI05  DLI06 DLI07 DLI08  DLI09  DLI10 DLI11   
2015-09-01 09:01    NaN   NaN  368.5  363.5   NaN   NaN  466.0    NaN   NaN   
2015-09-01 09:02    NaN   NaN  367.5    NaN  

 # 5. 计算加权平均过程

In [126]:
df =loadInstrumentData('ZZSR').head(10)
df 

close                                                  \
                 ZZSRX01 ZZSRX03 ZZSRX05 ZZSRX07 ZZSRX09 ZZSRX11 ZZSRX13   
2015-09-01 09:01    5233     NaN    5137     NaN    5168     NaN    5181   
2015-09-01 09:02    5222     NaN    5125     NaN    5149     NaN    5169   
2015-09-01 09:03    5220     NaN    5122     NaN    5149     NaN    5168   
2015-09-01 09:04    5221     NaN    5124     NaN    5150     NaN    5168   
2015-09-01 09:05    5216     NaN    5120     NaN    5146     NaN    5165   
2015-09-01 09:06    5223     NaN    5125     NaN    5155     NaN    5171   
2015-09-01 09:07    5229     NaN    5131     NaN    5158     NaN    5177   
2015-09-01 09:08    5230     NaN    5131     NaN    5158     NaN    5178   
2015-09-01 09:09    5224     NaN    5128     NaN    5154     NaN    5174   
2015-09-01 09:10    5226     NaN    5130     NaN    5155     NaN    5177   

                                                        date          \
                 ZZSRY01 ZZSRY03 ZZSRY09 ZZSRY11     ZZSRX01 ZZSRX03   
2015-09-01 09:01    5223     NaN    4910     NaN  2015-09-01     NaN   
2015-09-01 09:02     NaN     NaN     NaN     NaN  2015-09-01     NaN   
2015-09-01 09:03     NaN     NaN    4908     NaN  2015-09-01     NaN   
2015-09-01 09:04     NaN     NaN    4908     NaN  2015-09-01     NaN   
2015-09-01 09:05    5221     NaN     NaN     NaN  2015-09-01     NaN   
2015-09-01 09:06    5215     NaN     NaN     NaN  2015-09-01     NaN   
2015-09-01 09:07    5218     NaN    4934     NaN  2015-09-01     NaN   
2015-09-01 09:08     NaN     NaN     NaN     NaN  2015-09-01     NaN   
2015-09-01 09:09    5218     NaN     NaN     NaN  2015-09-01     NaN   
2015-09-01 09:10     NaN     NaN     NaN     NaN  2015-09-01     NaN   

                                                                      \
                     ZZSRX05 ZZSRX07     ZZSRX09 ZZSRX11     ZZSRX13   
2015-09-01 09:01  2015-09-01     NaN  2015-09-01     NaN  2015-09-01   
2015-09-01 09:02  2015-09-01     NaN  2015-09-01     NaN  2015-09-01   
2015-09-01 09:03  2015-09-01     NaN  2015-09-01     NaN  2015-09-01   
2015-09-01 09:04  2015-09-01     NaN  2015-09-01     NaN  2015-09-01   
2015-09-01 09:05  2015-09-01     NaN  2015-09-01     NaN  2015-09-01   
2015-09-01 09:06  2015-09-01     NaN  2015-09-01     NaN  2015-09-01   
2015-09-01 09:07  2015-09-01     NaN  2015-09-01     NaN  2015-09-01   
2015-09-01 09:08  2015-09-01     NaN  2015-09-01     NaN  2015-09-01   
2015-09-01 09:09  2015-09-01     NaN  2015-09-01     NaN  2015-09-01   
2015-09-01 09:10  2015-09-01     NaN  2015-09-01     NaN  2015-09-01   

                                                            high          \
                     ZZSRY01 ZZSRY03     ZZSRY09 ZZSRY11 ZZSRX01 ZZSRX03   
2015-09-01 09:01  2015-09-01     NaN  2015-09-01     NaN    5241     NaN   
2015-09-01 09:02         NaN     NaN         NaN     NaN    5232     NaN   
2015-09-01 09:03         NaN     NaN  2015-09-01     NaN    5222     NaN   
2015-09-01 09:04         NaN     NaN  2015-09-01     NaN    5221     NaN   
2015-09-01 09:05  2015-09-01     NaN         NaN     NaN    5222     NaN   
2015-09-01 09:06  2015-09-01     NaN         NaN     NaN    5227     NaN   
2015-09-01 09:07  2015-09-01     NaN  2015-09-01     NaN    5230     NaN   
2015-09-01 09:08         NaN     NaN         NaN     NaN    5230     NaN   
2015-09-01 09:09  2015-09-01     NaN         NaN     NaN    5230     NaN   
2015-09-01 09:10         NaN     NaN         NaN     NaN    5229     NaN   

                                                                          \
                 ZZSRX05 ZZSRX07 ZZSRX09 ZZSRX11 ZZSRX13 ZZSRY01 ZZSRY03   
2015-09-01 09:01    5142     NaN    5168     NaN    5185    5223     NaN   
2015-09-01 09:02    5134     NaN    5165     NaN    5177     NaN     NaN   
2015-09-01 09:03    5124     NaN    5150     NaN    5169     NaN     NaN   
2015-09-01 09:04    5124     NaN    5150     NaN    5169     NaN     NaN   
2015-09

In [127]:
df['close']

,ZZSRX01,ZZSRX03,ZZSRX05,ZZSRX07,ZZSRX09,ZZSRX11,ZZSRX13,ZZSRY01,ZZSRY03,ZZSRY09,ZZSRY11
2015-09-01 09:01,5233,NaN,5137,NaN,5168,NaN,5181,5223,NaN,4910,NaN
2015-09-01 09:02,5222,NaN,5125,NaN,5149,NaN,5169,NaN,NaN,NaN,NaN
2015-09-01 09:03,5220,NaN,5122,NaN,5149,NaN,5168,NaN,NaN,4908,NaN
2015-09-01 09:04,5221,NaN,5124,NaN,5150,NaN,5168,NaN,NaN,4908,NaN
2015-09-01 09:05,5216,NaN,5120,NaN,5146,NaN,5165,5221,NaN,NaN,NaN
2015-09-01 09:06,5223,NaN,5125,NaN,5155,NaN,5171,5215,NaN,NaN,NaN
2015-09-01 09:07,5229,NaN,5131,NaN,5158,NaN,5177,5218,NaN,4934,NaN
2015-09-01 09:08,5230,NaN,5131,NaN,5158,NaN,5178,NaN,NaN,NaN,NaN
2015-09-01 09:09,5224,NaN,5128,NaN,5154,NaN,5174,5218,NaN,NaN,NaN
2015-09-01 09:10,5226,NaN,5130,NaN,5155,NaN,5177,NaN,NaN,NaN,NaN


In [128]:
df['lot'] 

,ZZSRX01,ZZSRX03,ZZSRX05,ZZSRX07,ZZSRX09,ZZSRX11,ZZSRX13,ZZSRY01,ZZSRY03,ZZSRY09,ZZSRY11
2015-09-01 09:01,16842,NaN,1068,NaN,136,NaN,18238,160,NaN,32,NaN
2015-09-01 09:02,10000,NaN,556,NaN,38,NaN,10548,NaN,NaN,NaN,NaN
2015-09-01 09:03,4018,NaN,152,NaN,10,NaN,4260,NaN,NaN,38,NaN
2015-09-01 09:04,3606,NaN,480,NaN,18,NaN,4170,NaN,NaN,74,NaN
2015-09-01 09:05,4736,NaN,236,NaN,6,NaN,4980,2,NaN,NaN,NaN
2015-09-01 09:06,4240,NaN,420,NaN,10,NaN,4554,6,NaN,NaN,NaN
2015-09-01 09:07,4470,NaN,264,NaN,20,NaN,4900,2,NaN,10,NaN
2015-09-01 09:08,5246,NaN,528,NaN,8,NaN,5778,NaN,NaN,NaN,NaN
2015-09-01 09:09,4216,NaN,358,NaN,14,NaN,4590,4,NaN,NaN,NaN
2015-09-01 09:10,2440,NaN,112,NaN,2,NaN,2418,NaN,NaN,NaN,NaN


In [129]:
df['lot'].sum( axis = 1)

2015-09-01 09:01    36476
2015-09-01 09:02    21142
2015-09-01 09:03     8478
2015-09-01 09:04     8348
2015-09-01 09:05     9960
2015-09-01 09:06     9230
2015-09-01 09:07     9666
2015-09-01 09:08    11560
2015-09-01 09:09     9182
2015-09-01 09:10     4972
dtype: float64

In [130]:
r = df['lot'].div(df['lot'].sum( axis = 1), axis=0)
r

,ZZSRX01,ZZSRX03,ZZSRX05,ZZSRX07,ZZSRX09,ZZSRX11,ZZSRX13,ZZSRY01,ZZSRY03,ZZSRY09,ZZSRY11
2015-09-01 09:01,0.461728,NaN,0.029280,NaN,0.003728,NaN,0.500000,0.004386,NaN,0.000877,NaN
2015-09-01 09:02,0.472992,NaN,0.026298,NaN,0.001797,NaN,0.498912,NaN,NaN,NaN,NaN
2015-09-01 09:03,0.473933,NaN,0.017929,NaN,0.001180,NaN,0.502477,NaN,NaN,0.004482,NaN
2015-09-01 09:04,0.431960,NaN,0.057499,NaN,0.002156,NaN,0.499521,NaN,NaN,0.008864,NaN
2015-09-01 09:05,0.475502,NaN,0.023695,NaN,0.000602,NaN,0.500000,0.000201,NaN,NaN,NaN
2015-09-01 09:06,0.459372,NaN,0.045504,NaN,0.001083,NaN,0.493391,0.000650,NaN,NaN,NaN
2015-09-01 09:07,0.462446,NaN,0.027312,NaN,0.002069,NaN,0.506932,0.000207,NaN,0.001035,NaN
2015-09-01 09:08,0.453806,NaN,0.045675,NaN,0.000692,NaN,0.499827,NaN,NaN,NaN,NaN
2015-09-01 09:09,0.459159,NaN,0.038989,NaN,0.001525,NaN,0.499891,0.000436,NaN,NaN,NaN
2015-09-01 09:10,0.490748,NaN,0.022526,NaN,0.000402,NaN,0.486323,NaN,NaN,NaN,NaN


In [131]:
df['close']  * r

,ZZSRX01,ZZSRX03,ZZSRX05,ZZSRX07,ZZSRX09,ZZSRX11,ZZSRX13,ZZSRY01,ZZSRY03,ZZSRY09,ZZSRY11
2015-09-01 09:01,2416.223983,NaN,150.408926,NaN,19.268779,NaN,2590.500000,22.910407,NaN,4.307490,NaN
2015-09-01 09:02,2469.964999,NaN,134.779113,NaN,9.254659,NaN,2578.876738,NaN,NaN,NaN,NaN
2015-09-01 09:03,2473.927813,NaN,91.831092,NaN,6.073366,NaN,2596.801132,NaN,NaN,21.998585,NaN
2015-09-01 09:04,2255.261859,NaN,294.623862,NaN,11.104456,NaN,2581.523718,NaN,NaN,43.506469,NaN
2015-09-01 09:05,2480.218474,NaN,121.317269,NaN,3.100000,NaN,2582.500000,1.048394,NaN,NaN,NaN
2015-09-01 09:06,2399.297941,NaN,233.206934,NaN,5.585049,NaN,2551.325460,3.390033,NaN,NaN,NaN
2015-09-01 09:07,2418.128492,NaN,140.139044,NaN,10.672460,NaN,2624.384440,1.079661,NaN,5.104490,NaN
2015-09-01 09:08,2373.406574,NaN,234.357093,NaN,3.569550,NaN,2588.104152,NaN,NaN,NaN,NaN
2015-09-01 09:09,2398.647789,NaN,199.937269,NaN,7.858419,NaN,2586.436506,2.273143,NaN,NaN,NaN
2015-09-01 09:10,2564.650040,NaN,115.559131,NaN,2.073612,NaN,2517.696299,NaN,NaN,NaN,NaN


In [132]:
close = (df['close']  * r).sum(axis=1)
close

2015-09-01 09:01    5203.619585
2015-09-01 09:02    5192.875508
2015-09-01 09:03    5190.631989
2015-09-01 09:04    5186.020364
2015-09-01 09:05    5188.184137
2015-09-01 09:06    5192.805417
2015-09-01 09:07    5199.508587
2015-09-01 09:08    5199.437370
2015-09-01 09:09    5195.153126
2015-09-01 09:10    5199.979083
dtype: float64

In [133]:
df['lot']

,ZZSRX01,ZZSRX03,ZZSRX05,ZZSRX07,ZZSRX09,ZZSRX11,ZZSRX13,ZZSRY01,ZZSRY03,ZZSRY09,ZZSRY11
2015-09-01 09:01,16842,NaN,1068,NaN,136,NaN,18238,160,NaN,32,NaN
2015-09-01 09:02,10000,NaN,556,NaN,38,NaN,10548,NaN,NaN,NaN,NaN
2015-09-01 09:03,4018,NaN,152,NaN,10,NaN,4260,NaN,NaN,38,NaN
2015-09-01 09:04,3606,NaN,480,NaN,18,NaN,4170,NaN,NaN,74,NaN
2015-09-01 09:05,4736,NaN,236,NaN,6,NaN,4980,2,NaN,NaN,NaN
2015-09-01 09:06,4240,NaN,420,NaN,10,NaN,4554,6,NaN,NaN,NaN
2015-09-01 09:07,4470,NaN,264,NaN,20,NaN,4900,2,NaN,10,NaN
2015-09-01 09:08,5246,NaN,528,NaN,8,NaN,5778,NaN,NaN,NaN,NaN
2015-09-01 09:09,4216,NaN,358,NaN,14,NaN,4590,4,NaN,NaN,NaN
2015-09-01 09:10,2440,NaN,112,NaN,2,NaN,2418,NaN,NaN,NaN,NaN


In [134]:
df['lot'].sum(axis=1)

2015-09-01 09:01    36476
2015-09-01 09:02    21142
2015-09-01 09:03     8478
2015-09-01 09:04     8348
2015-09-01 09:05     9960
2015-09-01 09:06     9230
2015-09-01 09:07     9666
2015-09-01 09:08    11560
2015-09-01 09:09     9182
2015-09-01 09:10     4972
dtype: float64

# 6.定义指标计算函数

In [135]:
def calculateIndex(df):
    """
    通过一个品种的数据全集计算其加权平均
    """
    # 加权平均计算
    calColumns = ['open', 'high', 'low', 'close']
    r =  df['lot'].div(df['lot'].sum( axis = 1), axis=0)
    resultList = [(df[col]  * r).sum(axis=1) for col in calColumns]
    # 求和计算
    sumColumns = ['lot', 'volume']
    resultList += [df[col].sum(axis=1) for col in sumColumns]    
    # 合并数据
    result = pd.concat(resultList, axis=1, keys = calColumns+sumColumns)     
    # 拆分日期和时间
    result['date'] = Series(result.index, index=result.index).apply(lambda x:x.split()[0])
    result['time'] = Series(result.index, index=result.index).apply(lambda x:x.split()[1])
    # 字段排序
    result = result[ ['date', 'time'] + calColumns + sumColumns ]
    return result


In [136]:
result = calculateIndex(df)
#result['date'] = Series(result.index, index=result.index).apply(lambda x:x.split()[0])
#result['time'] = Series(result.index, index=result.index).apply(lambda x:x.split()[1])
result

,date,time,open,high,low,close,lot,volume
2015-09-01 09:01,2015-09-01,09:01,5184.753372,5209.459809,5182.694813,5203.619585,36476,1693230
2015-09-01 09:02,2015-09-01,09:02,5201.862170,5201.862170,5188.908618,5192.875508,21142,1632398
2015-09-01 09:03,2015-09-01,09:03,5192.101439,5192.119368,5188.660061,5190.631989,8478,1687684
2015-09-01 09:04,2015-09-01,09:04,5184.541926,5186.519885,5181.628174,5186.020364,8348,1689154
2015-09-01 09:05,2015-09-01,09:05,5192.682530,5193.158635,5185.186546,5188.184137,9960,1634478
2015-09-01 09:06,2015-09-01,09:06,5186.853738,5195.857205,5186.394366,5192.805417,9230,1634522
2015-09-01 09:07,2015-09-01,09:07,5192.551210,5200.534658,5190.551624,5199.508587,9666,1690370
2015-09-01 09:08,2015-09-01,09:08,5198.482353,5199.483045,5193.032007,5199.437370,11560,1634316
2015-09-01 09:09,2015-09-01,09:09,5199.649532,5200.108691,5192.159007,5195.153126,9182,1633502
2015-09-01 09:10,2015-09-01,09:10,5197.029767,5201.982703,5196.539019,5199.979083,4972,1633038


In [137]:
calculateIndex(df)    

,date,time,open,high,low,close,lot,volume
2015-09-01 09:01,2015-09-01,09:01,5184.753372,5209.459809,5182.694813,5203.619585,36476,1693230
2015-09-01 09:02,2015-09-01,09:02,5201.862170,5201.862170,5188.908618,5192.875508,21142,1632398
2015-09-01 09:03,2015-09-01,09:03,5192.101439,5192.119368,5188.660061,5190.631989,8478,1687684
2015-09-01 09:04,2015-09-01,09:04,5184.541926,5186.519885,5181.628174,5186.020364,8348,1689154
2015-09-01 09:05,2015-09-01,09:05,5192.682530,5193.158635,5185.186546,5188.184137,9960,1634478
2015-09-01 09:06,2015-09-01,09:06,5186.853738,5195.857205,5186.394366,5192.805417,9230,1634522
2015-09-01 09:07,2015-09-01,09:07,5192.551210,5200.534658,5190.551624,5199.508587,9666,1690370
2015-09-01 09:08,2015-09-01,09:08,5198.482353,5199.483045,5193.032007,5199.437370,11560,1634316
2015-09-01 09:09,2015-09-01,09:09,5199.649532,5200.108691,5192.159007,5195.153126,9182,1633502
2015-09-01 09:10,2015-09-01,09:10,5197.029767,5201.982703,5196.539019,5199.979083,4972,1633038


# 7. 定义完整的计算一个品种的加权平均加并保存到文件

In [138]:
def processInstrument(catalog):
    """
    计算品种的加权平均价，并保存到文件
    """
    df = loadInstrumentData(catalog)
    result = calculateIndex(df)
    filename = 'calculate_index_%s.csv' % catalog
    result.to_csv(join(path, filename), header=False, index=False)

In [139]:
processInstrument('DLI')

# 8. 循环处理所有品种

In [140]:
for catalog in instrumentList:
    print catalog,'...........',
    processInstrument(catalog)
    print u'完成'

DLAX ........... 完成
DLAY ........... 完成
DLB ........... 完成
DLBB ........... 完成
DLC ........... 完成
DLCS ........... 完成
DLFB ........... 完成
DLI ........... 完成
DLJ ........... 完成
DLJD ........... 完成
DLJM ........... 完成
DLL ........... 完成
DLM ........... 完成
DLP ........... 完成
DLPP ........... 完成
DLV ........... 完成
DLY ........... 完成
SQAG ........... 完成
SQAL ........... 完成
SQAU ........... 完成
SQBUX ........... 完成
SQBUY ........... 完成
SQCU ........... 完成
SQFU ........... 完成
SQHC ........... 完成
SQNI ........... 完成
SQPB ........... 完成
SQRB ........... 完成
SQRU ........... 完成
SQSN ........... 完成
SQWR ........... 完成
SQZN ........... 完成
ZZCF ........... 完成
ZZFG ........... 完成
ZZJR ........... 完成
ZZLR ........... 完成
ZZMA ........... 完成
ZZME ........... 完成
ZZOI ........... 完成
ZZPM ........... 完成
ZZRI ........... 完成
ZZRM ........... 完成
ZZRS ........... 完成
ZZSF ........... 完成
ZZSM ........... 完成
ZZSR ........... 完成
ZZTA ........... 完成
ZZTC ........... 完成
ZZWH ........... 完成
